In [11]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time

# Khởi động WebDriver (sử dụng Chrome ở đây)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Mở trang web
# Hàm để chuyển đổi chuỗi ngày thành đối tượng datetime
# def get_date_from_url(url):
#     # Lấy chuỗi ngày từ URL
#     date_str = url.split("week=")[-1]
#     return datetime.strptime(date_str, "%b%d.%Y")

# # Hàm để tạo URL mới với ngày giảm dần
# def create_new_url(base_url, date):
#     return base_url.split("?week=")[0] + "?week=" + date.strftime("%b%d.%Y").lower()

# # Ngày bắt đầu (từ URL ban đầu)
# start_date = get_date_from_url("https://www.forexfactory.com/calendar?week=jul14.2024")

url = "https://www.forexfactory.com/calendar?week=jul7.2024"
driver.get(url)

# Đợi trang tải và các phần tử cần thiết xuất hiện
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.calendar__cell.calendar__date'))
    )
except TimeoutException:
    print("Loading took too much time!")

# Cuộn trang xuống để tải thêm dữ liệu (nếu trang sử dụng lazy loading)
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollBy(0, 800);")  # Cuộn xuống 400px mỗi lần
    time.sleep(1)  # Chờ 1 giây trước khi cuộn tiếp
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # Nếu không có thêm dữ liệu mới, dừng lại
        break
    last_height = new_height

# Lấy các phần tử có các class CSS tương ứng
date_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__date')
time_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__time')
currency_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__currency')
impact_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__impact span')
event_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__event.event')
actual_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__actual')
forecast_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__forecast')
previous_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__previous')

# Mở tệp CSV để ghi dữ liệu
with open('calendar_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Time', 'Currency', 'Impact', 'Event', 'Actual', 'Forecast', 'Previous'])  # Tiêu đề cột

    # Duyệt qua từng phần tử và ghi vào tệp CSV
    current_date = ""
    current_time = ""
    for date_cell in date_cells:
        date_text = date_cell.text.replace('\n', ' ').strip()
        rowspan = date_cell.get_attribute('rowspan')  # Lấy thuộc tính rowspan để biết số hàng tiếp theo
        print(f"Date: {date_text}, Rowspan: {rowspan}")
        if rowspan:  # Nếu có thuộc tính rowspan
            for i in range(int(rowspan)):
                try:
                    time_text = time_cells[i].text.strip()
                    currency_text = currency_cells[i].text.strip()
                    impact_text = impact_cells[i].get_attribute('title').strip()
                    event_text = event_cells[i].text.strip()
                    actual_text = actual_cells[i].text.strip()
                    forecast_text = forecast_cells[i].text.strip()
                    previous_text = previous_cells[i].text.strip()
                    if forecast_text == "":  # Nếu không có dự báo, gán giá trị trước đó cho dự báo
                        forecast_text = 0
                    if date_text:  # Nếu gặp một ngày mới
                        current_date = date_text  # Cập nhật ngày hiện tại
                    if time_text:  # Nếu gặp một thời gian mới
                        current_time = time_text  # Cập nhật thời gian hiện tại

                    # Ghi dòng với cột Date giữ nguyên nếu không có giá trị mới
                    writer.writerow([current_date, current_time, currency_text, impact_text, event_text, actual_text, forecast_text, previous_text])

                except Exception as e:
                    print(f"Error: {e}")
        else:
            time_text = 0
            currency_text = 0
            impact_text = 0
            event_text = 0
            actual_text = 0
            forecast_text = 0
            previous_text = 0
            writer.writerow([date_text, time_text, currency_text, impact_text, event_text, actual_text, forecast_text, previous_text])
# Đóng trình duyệt
driver.quit()

print("Data has been saved to calendar_data.csv")


Date: Sun Jul 7, Rowspan: 1
Date: Mon Jul 8, Rowspan: 7
Date: Tue Jul 9, Rowspan: 10
Date: Wed Jul 10, Rowspan: 15
Date: Thu Jul 11, Rowspan: 24
Date: Fri Jul 12, Rowspan: 18
Date: Sat Jul 13, Rowspan: None
Data has been saved to calendar_data.csv


In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import os
import sqlite3

# Khởi động WebDriver (sử dụng Chrome ở đây)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

def get_calendar_data(url):
    driver.get(url)

    # Đợi trang tải và các phần tử cần thiết xuất hiện
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.calendar__cell.calendar__date'))
        )
    except TimeoutException:
        print("Loading took too much time!")
        return []

    # Cuộn trang xuống để tải thêm dữ liệu (nếu trang sử dụng lazy loading)
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollBy(0, 800);")  # Cuộn xuống 700px mỗi lần
        time.sleep(1)  # Chờ 1 giây trước khi cuộn tiếp
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Nếu không có thêm dữ liệu mới, dừng lại
            break
        last_height = new_height

    # Lấy các phần tử có các class CSS tương ứng
    date_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__date')
    time_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__time')
    currency_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__currency')
    impact_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__impact span')
    event_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__event.event')
    actual_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__actual')
    forecast_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__forecast')
    previous_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__previous')

    data = []
    current_date = ""
    current_time = ""

    for date_cell in date_cells:
        date_text = date_cell.text.replace('\n', ' ').strip()
        rowspan = date_cell.get_attribute('rowspan')  # Lấy thuộc tính rowspan để biết số hàng tiếp theo
        print(f"Date: {date_text}, Rowspan: {rowspan}")
        if rowspan:  # Nếu có thuộc tính rowspan
            for i in range(int(rowspan)):
                try:
                    time_text = time_cells[i].text.strip()
                    currency_text = currency_cells[i].text.strip()
                    impact_text = impact_cells[i].get_attribute('title').strip()
                    event_text = event_cells[i].text.strip()
                    actual_text = actual_cells[i].text.strip()
                    forecast_text = forecast_cells[i].text.strip()
                    previous_text = previous_cells[i].text.strip()
                    if forecast_text == "":  # Nếu không có dự báo, gán giá trị trước đó cho dự báo
                        forecast_text = 0
                    if date_text:  # Nếu gặp một ngày mới
                        current_date = date_text  # Cập nhật ngày hiện tại
                    if time_text:  # Nếu gặp một thời gian mới
                        current_time = time_text  # Cập nhật thời gian hiện tại


                    data.append([current_date, current_time, currency_text, impact_text, event_text, actual_text, forecast_text, previous_text])

                except Exception as e:
                    print(f"Error: {e}")
    
    return data

def create_database():
    conn = sqlite3.connect('news_data.db')  # Kết nối hoặc tạo cơ sở dữ liệu SQLite
    cursor = conn.cursor()

    # Tạo bảng nếu chưa tồn tại
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CalendarData (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            time TEXT,
            currency TEXT,
            impact TEXT,
            event TEXT,
            actual TEXT,
            forecast TEXT,
            previous TEXT
        )
    ''')

    conn.commit()
    conn.close()

# Gọi hàm tạo bảng
create_database()
def save_to_db(data):
    conn = sqlite3.connect('news_data.db')
    cursor = conn.cursor()

    # Chèn từng dòng dữ liệu vào bảng
    for row in data:
        cursor.execute('''
            INSERT INTO CalendarData (date, time, currency, impact, event, actual, forecast, previous)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', row)

    conn.commit()
    conn.close()


# Danh sách URL cho các tuần khác nhau
def generate_weekly_urls(base_url, start_date_str, num_weeks):
    # Chuyển đổi chuỗi ngày bắt đầu thành đối tượng datetime
    start_date = datetime.datetime.strptime(start_date_str, '%b%d.%Y')
    
    # Danh sách lưu trữ các URL
    urls = []
    
    for i in range(num_weeks):
        # Tạo đường dẫn tuần
        week_date = start_date + datetime.timedelta(weeks=i)
        week_str = week_date.strftime('%b%d.%Y').lower()  # Chuyển sang định dạng cần thiết
        
        # Tạo URL mới
        new_url = f"{base_url}?week={week_str}"
        urls.append(new_url)
    
    return urls



# Ví dụ sử dụng
base_url = "https://www.forexfactory.com/calendar"
start_date_str = ".2024"  # Ngày bắt đầu từ URL ban đầu
num_weeks = 3  # Số tuần cần tạo URL

weekly_urls = generate_weekly_urls(base_url, start_date_str, num_weeks)

# In ra danh sách các URL
for url in weekly_urls:
    week_data = get_calendar_data(url)
    # Xác định thời gian mới hơn hay cũ hơn để quyết định cách thêm vào CSV
    is_newer = True if "jul7.2024" in url else False
    # Gọi hàm lưu dữ liệu vào database
    save_to_db(week_data)

# Đóng trình duyệt
driver.quit()

print("Data has been saved to news_data.csv")


Date: Sun Jul 14, Rowspan: None
Date: Mon Jul 15, Rowspan: 18
Date: Tue Jul 16, Rowspan: 19
Date: Wed Jul 17, Rowspan: 20
Date: Thu Jul 18, Rowspan: 19
Date: Fri Jul 19, Rowspan: 17
Date: Sat Jul 20, Rowspan: 1
Loading took too much time!
Loading took too much time!
Data has been saved to news_data.csv


In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import sqlite3
import zipfile
import sqlite3


# Hàm khởi động WebDriver với proxy ngẫu nhiên
def start_driver_with_proxy():
    # Thông tin proxy
    proxy_host = "p.webshare.io"
    proxy_port = "80"
    proxy_user = "bebdhbcm-rotate"
    proxy_pass = "lzi9fgc0pdrc"

    # Tạo tiện ích mở rộng để xử lý xác thực proxy
    manifest_json = """
    {
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "Chrome Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {
            "scripts": ["background.js"]
        },
        "minimum_chrome_version":"22.0.0"
    }
    """

    background_js = f"""
    var config = {{
            mode: "fixed_servers",
            rules: {{
            singleProxy: {{
                scheme: "http",
                host: "{proxy_host}",
                port: parseInt({proxy_port})
            }},
            bypassList: ["localhost"]
            }}
        }};

    chrome.proxy.settings.set({{value: config, scope: "regular"}}, function() {{}});

    function callbackFn(details) {{
        return {{
            authCredentials: {{
                username: "{proxy_user}",
                password: "{proxy_pass}"
            }}
        }};
    }}

    chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {{urls: ["<all_urls>"]}},
            ['blocking']
    );
    """

    # Tạo file zip cho tiện ích mở rộng
    with zipfile.ZipFile("proxy_auth_plugin.zip", 'w') as zp:
        zp.writestr("manifest.json", manifest_json)
        zp.writestr("background.js", background_js)

    # Cấu hình ChromeOptions để sử dụng tiện ích mở rộng
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_extension("proxy_auth_plugin.zip")

    # Khởi tạo WebDriver với ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Khởi động WebDriver với proxy ngẫu nhiên
driver = start_driver_with_proxy()

def get_calendar_data(url):
    driver.get(url)

    # Đợi trang tải và các phần tử cần thiết xuất hiện
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.calendar__cell.calendar__date'))
        )
    except TimeoutException:
        print("Loading took too much time!")
        return []

    # Cuộn trang xuống để tải thêm dữ liệu (nếu trang sử dụng lazy loading)
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollBy(0, 700);")  # Cuộn xuống 700px mỗi lần
        time.sleep(1)  # Chờ 1 giây trước khi cuộn tiếp
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Nếu không có thêm dữ liệu mới, dừng lại
            break
        last_height = new_height

    # Lấy các phần tử có các class CSS tương ứng
    date_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__date')
    time_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__time')
    currency_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__currency')
    impact_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__impact span')
    event_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__event.event')
    actual_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__actual')
    forecast_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__forecast')
    previous_cells = driver.find_elements(By.CSS_SELECTOR, '.calendar__cell.calendar__previous')

    data = []
    current_date = ""
    current_time = ""

    for date_cell in date_cells:
        date_text = date_cell.text.replace('\n', ' ').strip()
        rowspan = date_cell.get_attribute('rowspan')  # Lấy thuộc tính rowspan để biết số hàng tiếp theo
        print(f"Date: {date_text}, Rowspan: {rowspan}")
        if rowspan:  # Nếu có thuộc tính rowspan
            for i in range(int(rowspan)):
                try:
                    time_text = time_cells[i].text.strip()
                    currency_text = currency_cells[i].text.strip()
                    impact_text = impact_cells[i].get_attribute('title').strip()
                    event_text = event_cells[i].text.strip()
                    actual_text = actual_cells[i].text.strip()
                    forecast_text = forecast_cells[i].text.strip()
                    previous_text = previous_cells[i].text.strip()
                    if forecast_text == "":  # Nếu không có dự báo, gán giá trị trước đó cho dự báo
                        forecast_text = 0
                    if date_text:  # Nếu gặp một ngày mới
                        current_date = date_text  # Cập nhật ngày hiện tại
                    if time_text:  # Nếu gặp một thời gian mới
                        current_time = time_text  # Cập nhật thời gian hiện tại

                    data.append([current_date, current_time, currency_text, impact_text, event_text, actual_text, forecast_text, previous_text])

                except Exception as e:
                    print(f"Error: {e}")
    
    return data


def create_database():
    conn = sqlite3.connect('news_data2016.db')  # Kết nối hoặc tạo cơ sở dữ liệu SQLite
    cursor = conn.cursor()

    # Tạo bảng nếu chưa tồn tại
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CalendarData (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            time TEXT,
            currency TEXT,
            impact TEXT,
            event TEXT,
            actual TEXT,
            forecast TEXT,
            previous TEXT
        )
    ''')

    conn.commit()
    conn.close()

# Gọi hàm tạo bảng
create_database()
def save_to_db(data):
    conn = sqlite3.connect('news_data2016.db')
    cursor = conn.cursor()

    # Chèn từng dòng dữ liệu vào bảng
    for row in data:
        cursor.execute('''
            INSERT INTO CalendarData (date, time, currency, impact, event, actual, forecast, previous)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', row)

    conn.commit()
    conn.close()


# Danh sách URL cho các tuần khác nhau
def generate_weekly_urls(base_url, start_date_str, num_weeks):
    # Chuyển đổi chuỗi ngày bắt đầu thành đối tượng datetime
    start_date = datetime.datetime.strptime(start_date_str, '%b%d.%Y')
    
    # Danh sách lưu trữ các URL
    urls = []
    
    for i in range(num_weeks):
        # Tạo đường dẫn tuần
        week_date = start_date + datetime.timedelta(weeks=i)
        week_str = week_date.strftime('%b%d.%Y').lower()  # Chuyển sang định dạng cần thiết
        
        # Tạo URL mới
        new_url = f"{base_url}?week={week_str}"
        urls.append(new_url)
    
    return urls

# Ví dụ sử dụng
base_url = "https://www.forexfactory.com/calendar"
start_date_str = "dec27.2015"  # Ngày bắt đầu từ URL ban đầu
num_weeks = 53  # Số tuần cần tạo URL

weekly_urls = generate_weekly_urls(base_url, start_date_str, num_weeks)

# In ra danh sách các URL
for url in weekly_urls:
    driver = start_driver_with_proxy()  # Mở trình duyệt mới với proxy mới
    week_data = get_calendar_data(url)
    # Xác định thời gian mới hơn hay cũ hơn để quyết định cách thêm vào CSV
    save_to_db(week_data)

# Đóng trình duyệt
driver.quit()

print("Data has been saved to news_data.db")


Date: Sun Dec 27, Rowspan: 4
Date: Mon Dec 28, Rowspan: 2
Date: Tue Dec 29, Rowspan: 3
Date: Wed Dec 30, Rowspan: 10
Date: Thu Dec 31, Rowspan: 11
Date: Fri Jan 1, Rowspan: 7
Date: Sat Jan 2, Rowspan: None
Date: Sun Jan 3, Rowspan: 6
Date: Mon Jan 4, Rowspan: 19
Date: Tue Jan 5, Rowspan: 15
Date: Wed Jan 6, Rowspan: 19
Date: Thu Jan 7, Rowspan: 18
Date: Fri Jan 8, Rowspan: 18
Date: Sat Jan 9, Rowspan: None
Date: Sun Jan 10, Rowspan: 1
Date: Mon Jan 11, Rowspan: 7
Date: Tue Jan 12, Rowspan: 15
Date: Wed Jan 13, Rowspan: 12
Date: Thu Jan 14, Rowspan: 21
Date: Fri Jan 15, Rowspan: 20
Date: Sat Jan 16, Rowspan: None
Date: Sun Jan 17, Rowspan: None
Date: Mon Jan 18, Rowspan: 8
Date: Tue Jan 19, Rowspan: 24
Date: Wed Jan 20, Rowspan: 21
Date: Thu Jan 21, Rowspan: 12
Date: Fri Jan 22, Rowspan: 19
Date: Sat Jan 23, Rowspan: 1
Date: Sun Jan 24, Rowspan: None
Date: Mon Jan 25, Rowspan: 10
Date: Tue Jan 26, Rowspan: 9
Date: Wed Jan 27, Rowspan: 16
Date: Thu Jan 28, Rowspan: 15
Date: Fri Jan 29, R